In [1]:
# Imports
import numpy as np

In [5]:
# Import the tools
from sys import path
path.append('/Users/reid/dev/PythonCode/tm/tools')
from tools import gen_inputs_outputs

In [48]:
# Define the size of the network
# Order by layers (input, layer1, layer2, ...)
# !!! No output layer yet, wait to gen inputs
layer_sizes = [4, 7]

In [49]:
# Gen inputs, outputs and size of the last layer
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

In [50]:
# Append the output layer size
layer_sizes = np.array(layer_sizes + [S], dtype=int)

In [51]:
# Network shape
layer_sizes

array([4, 7, 3])

In [52]:
# Inputs
inputs

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1]])

In [53]:
# Outputs
outputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0]])

In [75]:
# Intialize the weights and biases randomly
# Also create the list to store the outputs of each layer
weights_list = []
biases_list = []
n_list = []

In [76]:
for i in range(1, len(layer_sizes)):
    weights_list.append(np.random.rand(layer_sizes[i], layer_sizes[i-1]))
    biases_list.append(np.random.rand(layer_sizes[i], 1))
    n_list.append(np.empty((layer_sizes[i], 1)))

In [77]:
weights_list

[array([[0.03056803, 0.78871165, 0.99552296, 0.6453891 ],
        [0.13880434, 0.27118024, 0.276642  , 0.23735416],
        [0.08563308, 0.67475353, 0.54789107, 0.12007239],
        [0.44965829, 0.00335231, 0.6923589 , 0.24138516],
        [0.68770631, 0.18713529, 0.47686059, 0.27009779],
        [0.57573943, 0.47602118, 0.23506885, 0.15548008],
        [0.44728811, 0.30737562, 0.58289655, 0.79809042]]),
 array([[0.99813074, 0.07692107, 0.35851719, 0.85050821, 0.72920777,
         0.96698697, 0.89055572],
        [0.79757545, 0.48770538, 0.62967343, 0.30961213, 0.63433588,
         0.06212261, 0.39100406],
        [0.60374296, 0.0566358 , 0.46451235, 0.88523753, 0.48115666,
         0.51569581, 0.03831633]])]

In [78]:
biases_list

[array([[0.48407918],
        [0.76832438],
        [0.70627809],
        [0.19677455],
        [0.48081545],
        [0.31129566],
        [0.15757052]]),
 array([[0.55215158],
        [0.57362449],
        [0.60506571]])]

In [79]:
n_list

[array([[0.73656577],
        [0.8125669 ],
        [0.66117474],
        [0.76937536],
        [0.07154654],
        [0.04731709],
        [0.30214585]]),
 array([[0.78331681],
        [0.89954427],
        [0.55389619]])]

In [ ]:
# Define the transfer functions